In [1]:
import altair as alt
import numpy as np
import polars as pl


In [2]:
positions = pl.read_csv("positions.csv")

In [3]:
positions = positions.filter(
    (pl.col("x") > -90) & (pl.col("x") < 90) & (pl.col("y") > -180) & (pl.col("y") < 180)
).sort(
    ["k", "timestamp"]
).with_columns(
    (pl.col("timestamp")+"Z").str.strptime(pl.Datetime, fmt="%+", strict=False)
).with_columns(
    pl.internals.expr.ExprDateTimeNameSpace.seconds(pl.col("timestamp")-pl.col("timestamp").shift(1)).alias("time_diff_s")
)

In [4]:
# https://stackoverflow.com/questions/365826/calculate-distance-between-2-gps-coordinates
positions['distance_km'] = positions.select(
    ["x", "y", pl.col("x").shift(1).alias("prev_x"), pl.col("y").shift(1).alias("prev_y")]
).select(
    [np.deg2rad(pl.col("x")), np.deg2rad(pl.col("y")), np.deg2rad(pl.col("prev_x")), np.deg2rad(pl.col("prev_y")), ]
).with_columns([
    (pl.col("x") - pl.col("prev_x")).alias("d_lat"),
    (pl.col("y") - pl.col("prev_y")).alias("d_lon"),
]).with_columns([
    ((pl.col("d_lat") / 2).sin().pow(2) + (pl.col("d_lon") / 2).sin().pow(2) * pl.col("prev_x").cos() * pl.col("x").cos()).alias("a")
]).with_columns([
    ((pl.col("a").sqrt() / (1-pl.col("a")).sqrt()).arctan() * 2 * 6373).alias("distance_km")
])['distance_km']

In [5]:
positions = positions.with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("time_diff_s")).alias("time_diff_s")
).with_column(
    pl.when(pl.col("k").is_first())
    .then(None)
    .otherwise(pl.col("distance_km")).alias("distance_km")
).with_column(
    (pl.col("distance_km") / pl.col("time_diff_s") * 3600).alias("speed_km_h")
)